In [1]:
!pip install autokeras


     |████████████████████████████████| 166 kB 5.0 MB/s 
     |████████████████████████████████| 454.3 MB 16 kB/s 
     |████████████████████████████████| 97 kB 6.7 MB/s 
     |████████████████████████████████| 1.2 MB 46.2 MB/s 
     |████████████████████████████████| 4.0 MB 47.4 MB/s 
     |████████████████████████████████| 462 kB 72.3 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.39.0
    Uninstalling grpcio-1.39.0:
      Successfully uninstalled grpcio-1.39.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import autokeras as ak


## A Simple Example
The first step is to prepare your data. Here we use the MNIST dataset as an example


In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)


11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
[5 0 4]


The second step is to run the ImageClassifier.
It is recommended have more trials for more complicated datasets.
This is just a quick demo of MNIST, so we set max_trials to 1.
For the same reason, we set epochs to 10.
You can also leave the epochs unspecified for an adaptive number of epochs.


In [5]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=10)


# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))


Trial 1 Complete [00h 26m 47s]
val_loss: 0.042931631207466125

Best val_loss So Far: 0.042931631207466125
Total elapsed time: 00h 26m 47s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1875/1875 [==============================] - 172s 91ms/step - loss: 0.1558 - accuracy: 0.9529
Epoch 2/10
1875/1875 [==============================] - 166s 89ms/step - loss: 0.0720 - accuracy: 0.9784
Epoch 3/10
1875/1875 [==============================] - 167s 89ms/step - loss: 0.0568 - accuracy: 0.9825
Epoch 4/10
1875/1875 [==============================] - 167s 89ms/step - loss: 0.0484 - accuracy: 0.9851
Epoch 5/10
1875/1875 [==============================] - 169s 90ms/step - loss: 0.0460 - accuracy: 0.9848
Epoch 6/10
1875/1875 [==============================] - 171s 91ms/step - loss: 0.0395 - accuracy: 0.9872
Epoch 7/10
1875/1875 [==============================] - 171s 91ms/step - loss: 0.0348 - accuracy: 0.9891
Epoch 8/10
1875/1875 [==============================] - 170s 91ms/step - loss: 0.0346 - a

## Validation Data
By default, AutoKeras use the last 20% of training data as validation data. As
shown in the example below, you can use validation_split to specify the
percentage.


In [6]:
clf.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=10,
)


You can also use your own validation set instead of splitting it from the
training data with validation_data.


In [7]:
split = 50000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
clf.fit(
    x_train,
    y_train,
    # Use your own validation set.
    validation_data=(x_val, y_val),
    epochs=10,
)


## Customized Search Space
For advanced users, you may customize your search space by using AutoModel
instead of ImageClassifier. You can configure the ImageBlock for some
high-level configurations, e.g., block_type for the type of neural network to
search, normalize for whether to do data normalization, augment for whether to
do data augmentation. You can also do not specify these arguments, which would
leave the different choices to be tuned automatically. See the following
example for detail.


In [8]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=10)


Trial 1 Complete [00h 40m 53s]
val_loss: 0.18643873929977417

Best val_loss So Far: 0.18643873929977417
Total elapsed time: 00h 40m 53s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1563/1563 [==============================] - 233s 147ms/step - loss: 0.4529 - accuracy: 0.8647
Epoch 2/10
1563/1563 [==============================] - 229s 147ms/step - loss: 0.2442 - accuracy: 0.9264
Epoch 3/10
1563/1563 [==============================] - 224s 143ms/step - loss: 0.2037 - accuracy: 0.9374
Epoch 4/10
1563/1563 [==============================] - 221s 141ms/step - loss: 0.1822 - accuracy: 0.9437
Epoch 5/10
1563/1563 [==============================] - 223s 143ms/step - loss: 0.1682 - accuracy: 0.9480
Epoch 6/10
1563/1563 [==============================] - 225s 144ms/step - loss: 0.1580 - accuracy: 0.9512
Epoch 7/10
1563/1563 [==============================] - 224s 143ms/step - loss: 0.1502 - accuracy: 0.9536
Epoch 8/10
1563/1563 [==============================] - 223s 143ms/step - loss: 0.14

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


The usage of AutoModel is similar to the functional API of Keras. Basically, you are
building a graph, whose edges are blocks and the nodes are intermediate outputs of
blocks. To add an edge from input_node to output_node with output_node =
ak.[some_block]([block_args])(input_node).

You can even also use more fine grained blocks to customize the search space even
further. See the following example.


In [10]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ImageAugmentation(horizontal_flip=False)(output_node)
output_node = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=1)



Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
image_augmentat...|0                 |?                 
image_augmentat...|True              |?                 
image_augmentat...|0                 |?                 
image_augmentat...|0                 |?                 
image_augmentat...|0                 |?                 
res_net_block_1...|False             |?                 
res_net_block_1...|resnet152_v2      |?                 
res_net_block_1...|False             |?                 
classification_...|flatten           |?                 
classification_...|0                 |?                 
optimizer         |adam              |?                 
learning_rate     |0.001             |?                 

 327/1251 [======>.......................] - ETA: 1:44:53 - loss: 0.7740 - accuracy: 0.7706

KeyboardInterrupt: ignored

## Data Format
The AutoKeras ImageClassifier is quite flexible for the data format.

For the image, it accepts data formats both with and without the channel
dimension. The images in the MNIST dataset do not have the channel dimension.
Each image is a matrix with shape (28, 28). AutoKeras also accepts images of
three dimensions with the channel dimension at last, e.g., (32, 32, 3), (28,
28, 1).

For the classification labels, AutoKeras accepts both plain labels, i.e.
strings or integers, and one-hot encoded encoded labels, i.e. vectors of 0s and
1s.

So if you prepare your data in the following way, the ImageClassifier should
still work.


In [11]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the images to have the channel dimension.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

# One-hot encode the labels.
eye = np.eye(10)
y_train = eye[y_train]
y_test = eye[y_test]

print(x_train.shape)  # (60000, 28, 28, 1)
print(y_train.shape)  # (60000, 10)
print(y_train[:3])
# array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
#        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
#        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])


(60000, 28, 28, 1)
(60000, 10)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


We also support using tf.data.Dataset format for the training data.


In [12]:
train_set = tf.data.Dataset.from_tensor_slices(((x_train,), (y_train,)))
test_set = tf.data.Dataset.from_tensor_slices(((x_test,), (y_test,)))

clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the tensorflow Dataset to the classifier.
clf.fit(train_set, epochs=1)
# Predict with the best model.
predicted_y = clf.predict(test_set)
# Evaluate the best model with testing data.
print(clf.evaluate(test_set))


Trial 1 Complete [00h 02m 25s]
val_loss: 0.06271088123321533

Best val_loss So Far: 0.06271088123321533
Total elapsed time: 00h 02m 25s
INFO:tensorflow:Oracle triggered exit
1875/1875 [==============================] - 163s 87ms/step - loss: 0.1613 - accuracy: 0.9509
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets
313/313 [==============================] - 7s 23ms/step - loss: 0.0706 - accuracy: 0.9754
[0.07063596695661545, 0.9753999710083008]


## Reference
[ImageClassifier](/image_classifier),
[AutoModel](/auto_model/#automodel-class),
[ImageBlock](/block/#imageblock-class),
[Normalization](/block/#normalization-class),
[ImageAugmentation](/block/#image-augmentation-class),
[ResNetBlock](/block/#resnetblock-class),
[ImageInput](/node/#imageinput-class),
[ClassificationHead](/block/#classificationhead-class).
